### Step1. import library

In [ ]:
import traitlets , time
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot

### Step2. define robot motion

In [ ]:
robot = Robot()
speed = 0.7
sleep_time = 0.1

###  for button 
def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(speed)
    time.sleep(sleep_time)
    robot.stop()

def step_backward(change):
    robot.backward(speed)
    time.sleep(sleep_time)
    robot.stop()

def step_left(change):
    robot.left(speed)
    time.sleep(sleep_time)
    robot.stop()

def step_right(change):
    robot.right(speed)
    time.sleep(sleep_time)
    robot.stop()

### Step3. Create buttons ( or controller ) and link motion

creat buttons to control the car.

But these buttons wont do anything whule creating. We have to attach functions to move the car. 

In [ ]:
# create buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# display buttons
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])
display(controls_box)

# link buttons to actions
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

Or You can use Joystic to control you jetbot.
First we will find what button on your joystick.

In [ ]:
controller = widgets.Controller(index=0)
display(controller)

## for HJD-X (Vendor: 2563 Product: 0526)
controller_button = {}
controller_button['X'] = 3
controller_button['Y'] = 4
controller_button['B'] = 1
controller_button['A'] = 0
controller_button['L1'] = 6
controller_button['L2'] = 8
controller_button['R1'] = 7
controller_button['R2'] = 9
controller_button['start'] = 11
controller_button['select'] = 10
controller_button['analog'] = 12

All right! Now , we'll connect that to the left and right vertical axes using the dlink function. The dlink function, unlike the link function, allows us to attach a transform between the source and target.

In [ ]:
left_link = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda y: -y)
right_link = traitlets.dlink((controller.axes[5], 'value'), (robot.right_motor, 'value'), transform=lambda y: -y)

### Display live camera feed

So let's get started.  First, let's initialize and display our camera like we did in the *teleoperation* notebook.  

> Our neural network takes a 224x224 pixel image as input.  We'll set our camera to that size to minimize the filesize of our dataset (we've tested that it works for this task).
> In some scenarios it may be better to collect data in a larger image size and downscale to the desired size later.

In [ ]:
camera = Camera.instance(capture_flip = 2,width=224, height=224 , capture_device=1)

image = widgets.Image(format='jpeg', width=224, height=224 )  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

Awesome, next let's create a few directories where we'll store all our data. We'll create a folder dataset that will contain two sub-folders free and blocked, where we'll place the images for each scenario.

In [ ]:
import os

item = ['free' , 'green_light' , 'red_light' , 'parking' , 'zebra' , 'fence_open' , 'fence_close' , 'duck']
dataset_dir = {}
for name in item:
    dataset_dir[name]= 'dataset/' + str( item.index(name) ) + '_' + name
    print(dataset_dir[name])
# we have this "try/except" statement because these next functions can throw an error if the directories exist already

try:
    for name in dataset_dir.keys() :
        os.makedirs(dataset_dir[name])
except FileExistsError:
    print('Directories not created becasue they already exist')

If you refresh the Jupyter file browser on the left, you should now see those directories appear.  Next, let's create and display some buttons that we'll use to save snapshots
for each class label.  We'll also add some text boxes that will display how many images of each category that we've collected so far. This is useful because we want to make
sure we collect about as many images.  It also helps to know how many images we've collected overall.

In [ ]:
button_layout = widgets.Layout(width='128px', height='64px')
button_style = ['primary', 'success', 'info', 'warning', 'danger', '']

button = {}
count  = {}

for name in item:
    button[ name ] = widgets.Button(description= name, button_style='success', layout=button_layout)
    count[ name ]  = widgets.IntText(layout=button_layout, value=len(os.listdir( dataset_dir[name])))
    display(widgets.HBox([ button[ name], count[name] ] ) ) 
    button[ name ].button_style = button_style[ item.index(name) % 6 ]

## you can change the color of button
button[ 'green_light' ].button_style = 'primary' 
button[ 'green_light' ].button_style = 'success'    
button[ 'red_light' ].button_style = 'danger'
button[ 'parking' ].button_style = 'info'

Right now, these buttons wont do anything.  We have to attach functions to save images for each category to the buttons' ``on_click`` event.  We'll save the value
of the ``Image`` widget (rather than the camera), because it's already in compressed JPEG format!

To make sure we don't repeat any file names (even across different machines!) we'll use the ``uuid`` package in python, which defines the ``uuid1`` method to generate
a unique identifier.  This unique identifier is generated from information like the current time and the machine address.

If you want to test something , you can use it : 

test = widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False
    )

In [ ]:
from uuid import uuid1

class save_image():
    def __init__(self,item , dataset_dir):
        self.item = item
        self.dataset_dir = dataset_dir

    def save_name(self , kind_of_item):
        self.save_snapshot(kind_of_item)
        count[ kind_of_item ].value = len(os.listdir( dataset_dir[kind_of_item]))
            
    def save_snapshot(self,name):
        global image_path
        image_path = os.path.join(self.dataset_dir[name], str(uuid1()) + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image.value)    

            

# Notice
Now , you can choose keyboard or joystick to teleop Jetbot to collected data.
But remember , you just only can select one to do.

### Use Keyboard 

In [ ]:
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
# If you have four serval items to do , please add :  button[item[number]].on_click(lambda x: button_reaction.save_name(item[number]) )
# number is your item index , start at 0 .
button_reaction = save_image(item,dataset_dir)
button[item[0]].on_click(lambda x: button_reaction.save_name(item[0]) ) # " controller_button['X'] "
button[item[1]].on_click(lambda x: button_reaction.save_name(item[1]) ) # " controller_button['Y'] "
button[item[2]].on_click(lambda x: button_reaction.save_name(item[2]) ) # " controller_button['B'] "
button[item[3]].on_click(lambda x: button_reaction.save_name(item[3]) ) # " controller_button['A'] "
button[item[4]].on_click(lambda x: button_reaction.save_name(item[4]) ) # " controller_button['L2'] "
button[item[5]].on_click(lambda x: button_reaction.save_name(item[5]) ) # " controller_button['R2'] "
button[item[6]].on_click(lambda x: button_reaction.save_name(item[6]) ) # " controller_button['start'] "
button[item[7]].on_click(lambda x: button_reaction.save_name(item[7]) ) # " controller_button['select'] "

### Use Joystick controller
Below code is for controller. First we will show infromation of controller.
If you don't use controlle , please do not run below code.

Second is creating slider , link controller and slider.
If you don't use controlle , please do not run below code.

If you use controller , please run below code.

In [ ]:
# for controller 
# If you have four serval items to do , please add :  controller.buttons[controller_button['X']].observe(lambda x: button_reaction.save_name(item[0]) )
# number is your item index , start at 0 .

button_reaction = save_image(item,dataset_dir)
#controller.buttons[controller_button['X']].observe(lambda X: button_reaction.save_name , names='value' )
for b in controller.buttons:
    b.unobserve_all()

information = widgets.Textarea(
    value='Hello ! Default capture picture : free. And L1 is start auto-capture , R1 is stop auto-capture. ',
    placeholder='Type something',
    description='Status : ',
    disabled=False
    ) 

joystick_continus_picture_name = item[0]
capture_time = 0.5

class joystick():    
    def X(change):
        global joystick_continus_picture_name
        joystick_continus_picture_name = item[0]
        if change['new']:
            picture = button_reaction.save_name(joystick_continus_picture_name)      
            information.value = 'You Preess X and capture picture : {} '.format(joystick_continus_picture_name) 
            
    def Y(change):
        global joystick_continus_picture_name
        joystick_continus_picture_name = item[1]
        if change['new']:
            picture = button_reaction.save_name(joystick_continus_picture_name)
            information.value = 'You Preess Y and capture picture : {} '.format(joystick_continus_picture_name) 
            
    def B(change):
        global joystick_continus_picture_name
        joystick_continus_picture_name = item[2]
        if change['new']:
            picture = button_reaction.save_name(joystick_continus_picture_name)
            information.value = 'You Preess B and capture picture : {} '.format(joystick_continus_picture_name) 
            
    def A(change):
        global joystick_continus_picture_name
        joystick_continus_picture_name = item[3]
        if change['new']:
            picture = button_reaction.save_name(joystick_continus_picture_name)
            information.value = 'You Preess A and capture picture : {} '.format(joystick_continus_picture_name) 
            
    def L2(change):
        global joystick_continus_picture_name
        joystick_continus_picture_name = item[4]
        if change['new']:
            picture = button_reaction.save_name(joystick_continus_picture_name)
            information.value = 'You Preess A and capture picture : {} '.format(joystick_continus_picture_name)
            
    def R2(change):
        global joystick_continus_picture_name
        joystick_continus_picture_name = item[5]
        if change['new']:
            picture = button_reaction.save_name(joystick_continus_picture_name)
            information.value = 'You Preess A and capture picture : {} '.format(joystick_continus_picture_name)
            
    def start(change):
        global joystick_continus_picture_name
        joystick_continus_picture_name = item[6]
        if change['new']:
            picture = button_reaction.save_name(joystick_continus_picture_name)
            information.value = 'You Preess A and capture picture : {} '.format(joystick_continus_picture_name)
            
    def select(change):
        global joystick_continus_picture_name
        joystick_continus_picture_name = item[7]
        if change['new']:
            picture = button_reaction.save_name(joystick_continus_picture_name)
            information.value = 'You Preess A and capture picture : {} '.format(joystick_continus_picture_name)

    def L1(change):
        if change['new']:
            camera.observe(joystick.update, names='value')
            information.value = 'Start! you press L1 and capturing picture : {}'.format(joystick_continus_picture_name) 
            
    def R1(change):
        if change['new']:
            camera.unobserve(joystick.update, names='value')
            information.value = 'Stop! you press R1 and stop capturing picture. ' 
            
    def update(change):
        global joystick_continus_picture_name , capture_time
        picture = button_reaction.save_name(joystick_continus_picture_name)
        time.sleep(capture_time)

controller.buttons[controller_button['X']].observe(joystick.X , names='value')
controller.buttons[controller_button['Y']].observe(joystick.Y , names='value')
controller.buttons[controller_button['A']].observe(joystick.A , names='value')
controller.buttons[controller_button['B']].observe(joystick.B , names='value')
controller.buttons[controller_button['L2']].observe(joystick.L2 , names='value')
controller.buttons[controller_button['R2']].observe(joystick.R2 , names='value')
controller.buttons[controller_button['start']].observe(joystick.start , names='value')
controller.buttons[controller_button['select']].observe(joystick.select , names='value')
controller.buttons[controller_button['L1']].observe(joystick.L1 , names='value')
controller.buttons[controller_button['R1']].observe(joystick.R1 , names='value')

# Start Collect data!

In [ ]:
display(widgets.HBox([image, controls_box , information ]))

for text in item:
    display(widgets.HBox([button[text] , count[text]]))

Sometimes the folder will has checkpoint of ipython , we need delete it .

In [ ]:
!rm -rf dataset/0_free/.ipynb_checkpoints
!rm -rf dataset/1_green_light/.ipynb_checkpoints
!rm -rf dataset/2_red_light/.ipynb_checkpoints
!rm -rf dataset/3_parking/.ipynb_checkpoints
!rm -rf dataset/4_zebra/.ipynb_checkpoints
!rm -rf dataset/5_fence_open/.ipynb_checkpoints
!rm -rf dataset/6_fence_close/.ipynb_checkpoints
!rm -rf dataset/7_duck/.ipynb_checkpoints


Before shutdown this code , will need to unobserve camera 

In [ ]:
camera.unobserve(joystick.update, names='value')